In [3]:
from z3 import *

def compute_tax_fixed(big_category, sub_category, quantity):
    """
    計算固定稅額類別的應納稅額（針對水泥及油氣類）
    
    參數:
      big_category: 字串, "cement" 或 "oilgas"
      sub_category: 字串
         若 big_category == "cement":
           "white"         -> 白水泥或有色水泥 (600 NT$/ton)  (貨物稅條例 第7條)
           "bute1"         -> 卜特蘭一型 (320 NT$/ton)         (貨物稅條例 第7條)
           "bute2"         -> 卜特蘭高爐 (196 NT$/ton)         (原法條為280元，經調整)
           "other_cement"  -> 代水泥及其他水泥 (440 NT$/ton)    (貨物稅條例 第7條)
         若 big_category == "oilgas":
           "gasoline"      -> 汽油 (6830 NT$/unit)             (貨物稅條例 第10條)
           "diesel"        -> 柴油 (3990 NT$/unit)             (貨物稅條例 第10條)
           "kerosene"      -> 煤油 (4250 NT$/unit)             (貨物稅條例 第10條)
           "aviation"      -> 航空燃油 (610 NT$/unit)          (貨物稅條例 第10條)
           "fuel_oil"      -> 燃料油 (110 NT$/unit)            (貨物稅條例 第10條)
           "solvent_oil"   -> 溶劑油 (720 NT$/unit)            (貨物稅條例 第10條)
           "lpg"           -> 液化石油氣 (690 NT$/ton)         (貨物稅條例 第10條)
      quantity: 當月份出廠應稅數量 (整數)
      
    計算公式:
      應納稅額 = (單位稅額) * (數量)
      
    回傳:
      Z3 求解後的應納稅金額
    """
    s = Solver()
    tax_amount = Real('tax_amount')
    
    # 根據大種類決定 mapping，mapping 中之數值依據各法條規定
    if big_category == "cement":
        mapping = {
            "white": 600,         # 白水泥或有色水泥，每公噸600元 (貨物稅條例 第7條)
            "bute1": 320,          # 卜特蘭一型，每公噸320元 (貨物稅條例 第7條)
            "bute2": 196,          # 卜特蘭高爐，每公噸196元 (原法條為280元，經調整)
            "other_cement": 440    # 代水泥及其他水泥，每公噸440元 (貨物稅條例 第7條)
        }
    elif big_category == "oilgas":
        mapping = {
            "gasoline": 6830,      # 汽油，每公秉6830元 (貨物稅條例 第10條)
            "diesel": 3990,        # 柴油，每公秉3990元 (貨物稅條例 第10條)
            "kerosene": 4250,      # 煤油，每公秉4250元 (貨物稅條例 第10條)
            "aviation": 610,       # 航空燃油，每公秉610元 (貨物稅條例 第10條)
            "fuel_oil": 110,       # 燃料油，每公秉110元 (貨物稅條例 第10條)
            "solvent_oil": 720,    # 溶劑油，每公秉720元 (貨物稅條例 第10條)
            "lpg": 690             # 液化石油氣，每公噸690元 (貨物稅條例 第10條)
        }
    else:
        mapping = {}
    
    unit_tax = mapping.get(sub_category, 0)
    # 加入 constraint: tax_amount = unit_tax * quantity
    s.add(tax_amount == unit_tax * quantity)
    
    if s.check() == sat:
        m = s.model()
        return m.evaluate(tax_amount)
    else:
        return None

def compute_tax_others(big_category, sub_category, quantity, assessed_price, is_electric=False):
    """
    計算從價稅類別的應納稅額（針對橡膠輪胎、飲料品、平板玻璃、電器、車輛）
    
    參數:
      big_category: 字串，必須為下列之一：
           "rubber"    (橡膠輪胎)
           "beverage"  (飲料品)
           "flat_glass" (平板玻璃)
           "electrical" (電器)
           "vehicle"   (車輛)
      sub_category: 字串，依據大類別定義：
         若 big_category == "rubber":
           "bus_truck" -> 大客車、大貨車使用者 (從價徵收10%)
           "other"     -> 其他 (從價徵收15%)
           "exempt"    -> 免稅 (0%)
         若 big_category == "beverage":
           "diluted_juice" -> 稀釋天然果蔬汁 (從價徵收8%)
           "other"         -> 其他飲料品 (從價徵收15%)
           "exempt"        -> 免稅 (0%)
         若 big_category == "flat_glass":
           "regular"   -> 一般 (從價徵收10%)
           "exempt"    -> 免稅 (0%)
         若 big_category == "electrical":
           "refrigerator"           -> 電冰箱 (從價徵收13%)
           "color_tv"               -> 彩色電視機 (從價徵收13%)
           "central_aircon"         -> 冷暖氣機(中央系統型) (從價徵收15%)
           "non_central_aircon"     -> 冷暖氣機(非中央系統型) (從價徵收20%)
           "dehumidifier"           -> 除濕機 (從價徵收15%)
           "videorecorder"          -> 錄影機 (從價徵收13%)
           "non_portable_recordplayer" -> 電唱機(非手提32公分) (從價徵收10%)
           "portable_recordplayer"  -> 電唱機(手提32公分以下) (免稅 0%)
           "tape_recorder"          -> 錄音機 (從價徵收10%)
           "stereo"                 -> 音響組合 (從價徵收10%)
           "electric_oven"          -> 電烤箱 (從價徵收15%)
         若 big_category == "vehicle":
           "compact_car"   -> 小客車(2000cc以下) (從價徵收25%)
           "large_car"     -> 小客車(2000cc以上) (從價徵收30%)
           "truck_bus_others" -> 貨車、大客車及其他 (從價徵收15%)
           "motorcycle"    -> 機車 (從價徵收17%)
      quantity: 當月份出廠應稅數量 (整數)
      assessed_price: 每單位完稅價格 (例如由銷售價格/(1+稅率) 得到)
      is_electric: (僅針對 vehicle 類) 布林值，若為 True 則稅率依規定減半
      
    計算公式:
      應納稅額 = assessed_price * (從價稅率) * quantity
      (若為車輛且 is_electric 為 True，則稅率折半)
      
    回傳:
      Z3 求解後的應納稅金額
    """
    s = Solver()
    tax_amount = Real('tax_amount')
    
    # 根據大類別與子類別決定稅率
    if big_category == "rubber":
        mapping = {"bus_truck": 0.10, "other": 0.15, "exempt": 0.0}
        rate = mapping.get(sub_category, 0)
    elif big_category == "beverage":
        mapping = {"diluted_juice": 0.08, "other": 0.15, "exempt": 0.0}
        rate = mapping.get(sub_category, 0)
    elif big_category == "flat_glass":
        mapping = {"regular": 0.10, "exempt": 0.0}
        rate = mapping.get(sub_category, 0)
    elif big_category == "electrical":
        mapping = {
            "refrigerator": 0.13,
            "color_tv": 0.13,
            "central_aircon": 0.15,
            "non_central_aircon": 0.20,
            "dehumidifier": 0.15,
            "videorecorder": 0.13,
            "non_portable_recordplayer": 0.10,
            "portable_recordplayer": 0.0,
            "tape_recorder": 0.10,
            "stereo": 0.10,
            "electric_oven": 0.15
        }
        rate = mapping.get(sub_category, 0)
    elif big_category == "vehicle":
        mapping = {
            "compact_car": 0.25,
            "large_car": 0.30,
            "truck_bus_others": 0.15,
            "motorcycle": 0.17
        }
        rate = mapping.get(sub_category, 0)
        # 若為電動車或油電混合車輛，則稅率折半
        if is_electric:
            rate = rate / 2.0
    else:
        rate = 0
    
    # 加入 constraint: tax_amount = assessed_price * rate * quantity
    s.add(tax_amount == assessed_price * rate * quantity)
    
    if s.check() == sat:
        m = s.model()
        return m.evaluate(tax_amount)
    else:
        return None

# 測試範例
if __name__ == '__main__':
    # 範例1：水泥類，選擇白水泥，出廠數量10公噸
    tax1 = compute_tax_fixed("cement", "white", 10)
    print("水泥類 Tax:", tax1)  # 預期 600 * 10 = 6000
    
    # 範例2：油氣類，選擇汽油，出廠數量5 (單位：公秉)
    tax2 = compute_tax_fixed("oilgas", "gasoline", 5)
    print("油氣類 Tax:", tax2)  # 預期 6830 * 5 = 34150
    
    # 範例3：其他類目 - 電器，選擇冷暖氣機(中央系統型)，完稅價格2000元，出廠數量2台
    tax3 = compute_tax_others("electrical", "central_aircon", 2, 2000)
    print("電器 Tax:", tax3)  # 預期 2000 * 0.15 * 2 = 600
    
    # 範例4：其他類目 - 車輛，選擇小客車(2000cc以下) (25%)，非電動車，完稅價格800000元，出廠數量3輛
    tax4 = compute_tax_others("vehicle", "compact_car", 3, 800000, is_electric=False)
    print("車輛 Tax:", tax4)  # 預期 800000 * 0.25 * 3 = 600000
    
    # 範例5：其他類目 - 車輛，同上但若為則稅率減半 (0.25/2 = 0.125)
    tax5 = compute_tax_others("vehicle", "compact_car", 3, 800000, is_electric=True)
    print("電動車 Tax:", tax5)  # 預期 800000 * 0.125 * 3 = 300000


水泥類 Tax: 6000
油氣類 Tax: 34150
電器 Tax: 600
車輛 Tax: 600000
電動車 Tax: 300000


### 隨機參數 直接手動作

In [4]:
import math, random
from z3 import Real, Solver, sat

# ------------------------------
# 計算器 function（固定稅額）
# ------------------------------
def compute_tax_fixed(big_category, sub_category, quantity):
    """
    計算固定稅額類別的應納稅額（針對水泥及油氣類）
    計算公式: 應納稅額 = (單位稅額) * (數量)
    """
    s = Solver()
    tax_amount = Real('tax_amount')
    
    if big_category == "cement":
        mapping = {
            "white": 600,         # 白水泥或有色水泥，每公噸600元
            "bute1": 320,          # 卜特蘭一型
            "bute2": 196,          # 卜特蘭高爐
            "other_cement": 440    # 代水泥及其他水泥
        }
    elif big_category == "oilgas":
        mapping = {
            "gasoline": 6830,      # 汽油，每公秉6830元
            "diesel": 3990,        # 柴油，每公秉3990元
            "kerosene": 4250,      # 煤油，每公秉4250元
            "aviation": 610,       # 航空燃油，每公秉610元
            "fuel_oil": 110,       # 燃料油，每公秉110元
            "solvent_oil": 720,    # 溶劑油，每公秉720元
            "lpg": 690             # 液化石油氣，每公噸690元
        }
    else:
        mapping = {}
    
    unit_tax = mapping.get(sub_category, 0)
    s.add(tax_amount == unit_tax * quantity)
    
    if s.check() == sat:
        m = s.model()
        return m.evaluate(tax_amount)
    else:
        return None

# ------------------------------
# 計算器 function（從價稅）
# ------------------------------
def compute_tax_others(big_category, sub_category, quantity, assessed_price, is_electric=False):
    """
    計算從價稅類別的應納稅額（針對橡膠輪胎、飲料品、平板玻璃、電器、車輛）
    計算公式: 應納稅額 = assessed_price * (從價稅率) * quantity
      若為車輛且 is_electric 為 True，則稅率折半
    """
    s = Solver()
    tax_amount = Real('tax_amount')
    
    if big_category == "rubber":
        mapping = {"bus_truck": 0.10, "other": 0.15, "exempt": 0.0}
        rate = mapping.get(sub_category, 0)
    elif big_category == "beverage":
        mapping = {"diluted_juice": 0.08, "other": 0.15, "exempt": 0.0}
        rate = mapping.get(sub_category, 0)
    elif big_category == "flat_glass":
        mapping = {"regular": 0.10, "exempt": 0.0}
        rate = mapping.get(sub_category, 0)
    elif big_category == "electrical":
        mapping = {
            "refrigerator": 0.13,
            "color_tv": 0.13,
            "central_aircon": 0.15,
            "non_central_aircon": 0.20,
            "dehumidifier": 0.15,
            "videorecorder": 0.13,
            "non_portable_recordplayer": 0.10,
            "portable_recordplayer": 0.0,
            "tape_recorder": 0.10,
            "stereo": 0.10,
            "electric_oven": 0.15
        }
        rate = mapping.get(sub_category, 0)
    elif big_category == "vehicle":
        mapping = {
            "compact_car": 0.25,
            "large_car": 0.30,
            "truck_bus_others": 0.15,
            "motorcycle": 0.17
        }
        rate = mapping.get(sub_category, 0)
        if is_electric:
            rate = rate / 2.0
    else:
        rate = 0
    
    s.add(tax_amount == assessed_price * rate * quantity)
    
    if s.check() == sat:
        m = s.model()
        tax_value = int(float(m[tax_amount].as_decimal(20)))
        return (tax_value)
    else:
        return None
    
def round_half_up(x: float) -> int:
    """
    正常四捨五入函式：
      當小數部分 >= 0.5 時，往上取整，
      適用於非負數情形（稅額通常為正）。
    """
    return int(math.floor(x + 0.5))
# ------------------------------
# 隨機產生 100 組參數，並計算本地結果
# ------------------------------

# 固定稅額參數設定：
fixed_big_categories = {
    "cement": ["white", "bute1", "bute2", "other_cement"],
    "oilgas": ["gasoline", "diesel", "kerosene", "aviation", "fuel_oil", "solvent_oil", "lpg"]
}

# 從價稅參數設定：
others_big_categories = {
    "rubber": ["bus_truck", "other", "exempt"],
    "beverage": ["diluted_juice", "other", "exempt"],
    "flat_glass": ["regular", "exempt"],
    "electrical": ["refrigerator", "color_tv", "central_aircon", "non_central_aircon", 
                   "dehumidifier", "videorecorder", "non_portable_recordplayer", "portable_recordplayer",
                   "tape_recorder", "stereo", "electric_oven"],
    "vehicle": ["compact_car", "large_car", "truck_bus_others", "motorcycle"]
}

# 儲存測試資料
random_tests = []

for i in range(10):
    test_case = {}
    # 隨機決定使用固定稅額或從價稅 (0 固定, 1 從價)
    tax_type = random.choice([0, 1])
    test_case["tax_type"] = "fixed" if tax_type == 0 else "others"
    if tax_type == 0:
        # 固定稅額參數
        big_category = random.choice(list(fixed_big_categories.keys()))
        sub_category = random.choice(fixed_big_categories[big_category])
        quantity = random.randint(1, 50)
        test_case["big_category"] = big_category
        test_case["sub_category"] = sub_category
        test_case["quantity"] = quantity
        # 本地計算結果
        local_result = compute_tax_fixed(big_category, sub_category, quantity)
        print(f"本地計算結果: {local_result}, big_category: {big_category}, sub_category: {sub_category}, quantity: {quantity}")
    else:
        # 從價稅參數
        big_category = random.choice(list(others_big_categories.keys()))
        sub_category = random.choice(others_big_categories[big_category])
        quantity = random.randint(1, 50)
        assessed_price = random.randint(100, 1000000)
        is_electric = False
        if big_category == "vehicle":
            is_electric = random.choice([True, False])
        test_case["big_category"] = big_category
        test_case["sub_category"] = sub_category
        test_case["quantity"] = quantity
        test_case["assessed_price"] = assessed_price
        test_case["is_electric"] = is_electric
        # 本地計算結果
        local_result = compute_tax_others(big_category, sub_category, quantity, assessed_price, is_electric)
        print(f"本地計算結果: {local_result}, big_category: {big_category}, sub_category: {sub_category}, quantity: {quantity}, assessed_price: {assessed_price}, is_electric: {is_electric}")


本地計算結果: 4254529, big_category: electrical, sub_category: electric_oven, quantity: 30, assessed_price: 945451, is_electric: False
本地計算結果: 3960, big_category: cement, sub_category: other_cement, quantity: 9
本地計算結果: 12800, big_category: cement, sub_category: bute1, quantity: 40
本地計算結果: 0, big_category: beverage, sub_category: exempt, quantity: 7, assessed_price: 591207, is_electric: False
本地計算結果: 2592839, big_category: electrical, sub_category: electric_oven, quantity: 22, assessed_price: 785709, is_electric: False
本地計算結果: 733810, big_category: vehicle, sub_category: compact_car, quantity: 4, assessed_price: 733810, is_electric: False
本地計算結果: 2217789, big_category: electrical, sub_category: refrigerator, quantity: 47, assessed_price: 362977, is_electric: False
本地計算結果: 152022, big_category: beverage, sub_category: other, quantity: 11, assessed_price: 92135, is_electric: False
本地計算結果: 648631, big_category: electrical, sub_category: refrigerator, quantity: 16, assessed_price: 311842, is_elec